In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_differential_filter():
    # To do
    filter_x = np.tile(np.array([1,0,-1]),(3,1))
    filter_y = np.tile(np.vstack(np.array([1,0,-1])),3)
    
    return filter_x, filter_y

def filter_image(im, filter):
    # To do
    im_zero_pad = np.pad(im, ((1,1),(1,1)), 'constant')
    height,width = im_zero_pad.shape
    im_filtered = np.zeros(im.shape)
    
    for i in range(1,height-1,1):
        for j in range(1,width-1,1):
            cell = np.array([[im_zero_pad[i-1,j-1],im_zero_pad[i-1,j],im_zero_pad[i-1,j+1]],
                            [im_zero_pad[i,j-1],im_zero_pad[i,j],im_zero_pad[i,j+1]],
                            [im_zero_pad[i+1,j-1],im_zero_pad[i+1,j],im_zero_pad[i+1,j+1]]])
        
            
            im_filtered[i-1,j-1] = np.sum(cell*filter)
    return im_filtered


def get_gradient(im_dx, im_dy):
    # To do
    grad_mag = np.sqrt(im_dx*im_dx+im_dy*im_dy)
    grad_angle = np.zeros(filter_x.shape)
    for i in range(im_dx.shape[0]):
        for j in range(im_dx.shape[1]):
            grad_angle[i,j] = np.arctan2(im_dy[i,j],im_dx[i,j])
        
    return grad_mag, grad_angle % np.pi 


def build_histogram(grad_mag, grad_angle, cell_size):
    # To do
    angle_im = grad_angle*180/np.pi

    angle_im[np.where(angle_im==180.0)] = 179.9

    oder_im = (angle_im // 15)%11

    for i in range(6):
        oder_im[np.where(np.logical_and(oder_im>=2*i-1, oder_im<2*i+1))] = i
    
    ori_histo = np.zeros((6,oder_im[0]//cell_size,oder_im[1]//cell_size))
    
    for i in range(oder_im[0]):
        for j in range(oder_im[1]):
            ori_histo[oder_im[i,j],i//cell_size,j//cell_size] =grad_mag[i,j] + ori_histo[oder_im[i,j],i//cell_size,j//cell_size]
    
    return ori_histo


def get_block_descriptor(ori_histo, block_size):
    # To do
    return ori_histo_normalized


def extract_hog(im):
    # convert grey-scale image to double format
    im = im.astype('float') / 255.0
    
    # To do
    return hog






In [4]:
def visualize_hog(im, ori_histo, cell_size):
    norm_constant = 1e-3
    num_bins = ori_histo.shape[2]
    height, width = im.shape
    max_len = cell_size / 3
    angles = np.arange(0, np.pi, np.pi/num_bins)
    mesh_x, mesh_y = np.meshgrid(np.r_[cell_size/2:width:cell_size], np.r_[cell_size/2:height:cell_size])

    bin_ave = np.sqrt(np.sum(ori_histo ** 2, axis=2) + norm_constant ** 2)  # (ori_histo.shape[0], ori_histo.shape[1])
    histo_normalized = ori_histo / np.expand_dims(bin_ave, axis=2) * max_len  # same dims as ori_histo
    mesh_u = histo_normalized * np.sin(angles).reshape((1, 1, num_bins))  # expand to same dims as histo_normalized
    mesh_v = histo_normalized * -np.cos(angles).reshape((1, 1, num_bins))  # expand to same dims as histo_normalized
    plt.imshow(im, cmap='gray', vmin=0, vmax=1)
    for i in range(num_bins):
        plt.quiver(mesh_x - mesh_u[:, :, i], mesh_y - mesh_v[:, :, i], 2 * mesh_u[:, :, i], 2 * mesh_v[:, :, i],
                   color='white', headaxislength=0, headlength=0, scale_units='xy', scale=1, width=0.002, angles='xy')
    plt.show()


if __name__=='__main__':
    im = cv2.imread('einstein.jpg', 0)
    hog = extract_hog(im)

NameError: name 'hog' is not defined